In [1]:
import pandas as pd

ds = pd.read_parquet('./results/ds.parquet')
elapse_time = pd.read_csv('./results/elapse_time.csv')

In [2]:
from bertopic import BERTopic
from gensim.models.ldamulticore import LdaMulticore
from utils import e_variant

def load_ldas():
    r = {} # result
    for v in ds:
        r[v] = LdaMulticore.load(f'./results/models_lda/{v}')
    return r

def load_bertopics():
    path_prefix = './results/models_bertopic/'
    r = {}
    for v in ds:
        r[v] = {}
        r[v] = BERTopic.load(f'{path_prefix}{v}')
    return r

d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopython'

In [3]:
ldas = load_ldas()
bertopics = load_bertopics()

In [4]:
from time import time
from gensim.corpora.dictionary import Dictionary
from utils import get_topics_lda, get_topics_bertopic, get_coherence, get_diversity

def evaluate_models(models, lda=False):
    r = []
    r_time = []
    for v in ds:
        t_start = time()
        docs = ds[v].dropna()
        if lda:
            topics = get_topics_lda(models[v], models[v].id2word)
            texts = docs
            dictionary = models[v].id2word
        else:
            topics = get_topics_bertopic(bertopics[v])
            analyzer = models[v].vectorizer_model.build_analyzer()
            texts = [' '.join(doc) for doc in docs]
            texts = [analyzer(doc) for doc in texts]
            dictionary = Dictionary(texts)
        c = get_coherence(
            topics=topics,
            texts=texts,
            dictionary=dictionary,
        )
        d = get_diversity(topics)
        t_end = time()
        r_time.append(t_end - t_start)
        r.append({
            'variant': v,
            'coherence': c,
            'diversity': d,
            'score': c * d
        })
    algorithm = 'lda' if lda else 'bertopic'
    elapse_time[f'{algorithm}_evaluation'] = pd.Series(r_time)
    return pd.DataFrame(r)

In [5]:
import pandas as pd

evaluation_lda = pd.DataFrame(evaluate_models(ldas, lda=True))

In [6]:
evaluation_bertopic = pd.DataFrame(evaluate_models(bertopics))

In [7]:
elapse_time

,variant,nlp,preprocessing,lda_training,bertopic_training,lda_evaluation,bertopic_evaluation
0,dCL,26.75152,0.015996,22.314663,57.348722,5.392719,4.019900
1,DCL,26.75152,0.015996,21.944010,11.408381,4.875149,3.980268
2,dLWN,26.75152,0.097994,26.633091,35.589177,3.683330,3.687485
3,DLWN,26.75152,0.184994,18.792636,12.978124,3.987625,3.776740


In [8]:
evaluation_lda

,variant,coherence,diversity,score
0,dCL,0.454724,0.129688,0.058972
1,DCL,0.340701,0.129688,0.044185
2,dLWN,0.291858,0.500000,0.145929
3,DLWN,0.348887,0.500000,0.174444


In [9]:
evaluation_bertopic

,variant,coherence,diversity,score
0,dCL,0.405232,1.0,0.405232
1,DCL,0.343653,1.0,0.343653
2,dLWN,0.307534,1.0,0.307534
3,DLWN,0.342356,1.0,0.342356


In [10]:
elapse_time.to_csv('./results/elapse_time.csv', index=False)
evaluation_lda.to_csv('./results/evaluation_lda.csv', index=False)
evaluation_bertopic.to_csv('./results/evaluation_bertopic.csv', index=False)